In [ ]:
!pip install transliterate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.6 MB/s eta 0:00:00


In [13]:

import random
import datetime
import os
import argparse
from transliterate import translit
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql.functions import col, when, count

In [14]:
# Список из 25 имен
prepared_list_names = [
    "Алексей", "Ольга", "Дмитрий", "Мария", "Иван",
    "Анна", "Сергей", "Елена", "Андрей", "Наталья",
    "Владимир", "Татьяна", "Михаил", "Светлана", "Кирилл",
    "Юлия", "Александр", "Екатерина", "Николай", "Ирина",
    "Василий", "Евгений", "Людмила", "Павел", "Роман"
]

# Список из 40 городов
prepared_list_cities = [
    "Москва", "Санкт-Петербург", "Новосибирск", "Екатеринбург", "Казань",
    "Нижний Новгород", "Челябинск", "Самара", "Омск", "Ростов-на-Дону",
    "Уфа", "Красноярск", "Воронеж", "Пермь", "Волгоград",
    "Краснодар", "Тольятти", "Ижевск", "Ульяновск", "Барнаул",
    "Тюмень", "Иркутск", "Саратов", "Хабаровск", "Ярославль",
    "Владивосток", "Махачкала", "Томск", "Оренбург", "Кемерово",
    "Новокузнецк", "Рязань", "Астрахань", "Пенза", "Липецк",
    "Тула", "Киров", "Брянск", "Чебоксары", "Калининград"
]


In [15]:
current_date = datetime.date.today()

In [16]:
# ф-ция генерации даты рождения
def generate_birth_date(age: int) -> datetime.date:
    # Расчет даты рождения для случайного возраста
    random_date_of_birth = current_date - datetime.timedelta(days=age * 365)

    return random_date_of_birth

In [17]:
# ф-ция генерации даты регистрации.
# Ограничения:
# регистрация возможна после достижения человеком пяти лет и до текущей даты.

def generate_registration_date(birth_date: datetime.date) -> datetime.date:
    # toordinal() переводит дату в дни, а fromordinal() обратно в дату
    start_date = birth_date.toordinal() + 5*365
    end_date = current_date.toordinal()
    random_registr_date = datetime.date \
        .fromordinal(random.randint(start_date, end_date))

    return random_registr_date

In [18]:
# ф-ции выбора случайного имени и города из отфильтрованного списка

def random_name() -> str:
  return random.choice([name for name in prepared_list_names if len(name) >= 5])

def random_city() -> str:
  return random.choice([city for city in prepared_list_cities if len(city) >= 7])

In [19]:
# ф-ция удаляет все файлы, кроме .csv и переименовывает его в формат "текущий год-месяц-день-dev.csv"

def rename_csv(output_path: str) -> str:
    files_list = os.listdir(output_path)
    new_filename = output_path + f'/{current_date}-dev.csv'

    for file in files_list:
        if file.startswith('part-') and file.endswith('.csv'):
            old_name = output_path + '/' + file
            os.rename(old_name, new_filename)
        else:
            os.remove(os.path.join(output_path, file))

    return new_filename

In [20]:
# ф-ция генерации данных

def generate_data(num: int) -> list[tuple]:
    data = []

    # Рассчитываем количество NULL значений (не более 5% в каждом столбце)
    number_null_in_table = int(num * 0.05)

    # Для каждого столбца создаем свой счетчик
    null_counts = {
        'name': 0,
        'email': 0,
        'city': 0,
        'age': 0,
        'salary': 0,
        'registration_date': 0
    }

    # Генерация данных
    for i in range(0, num):
        name = random_name()
        city = random_city()
        age = random.randint(18, 95) # Генерация случайного возраста от 18 до 95 лет
        birth_date = generate_birth_date(age)
        registration_date = generate_registration_date(birth_date)
        #во время преобразования символов из ru в en translit заменяет "ь" на "'' (Ольга -> Ol'ga)
        # с помощью .replace("'", "") убираем "'" (Ольга -> Olga)
        email = translit(name.lower(), 'ru', reversed=True).replace("'", "")\
        + str(birth_date.year) + '_' + str(birth_date.day) + '@'\
        + random.choice(['ru', 'com'])
        salary = random.choice(range(100, 500)) * 10**3

        # промежуточная запись в словарь
        dict_data ={
            'name':name,
            'city':city,
            'age':age,
            'registration_date':registration_date,
            'email':email,
            'salary':salary
        }

        # Проверяем для каждого ключа (столбца) условие для вставки NULL значений
        for k, v in dict_data.items():
            #1. счетчик для столбца <= допустимому кол-ву NULL значений по столбцу
            #2. вероятность вставки NULL (если random.random() возвращает число меньше 0.05)
            if null_counts[k] <= number_null_in_table and random.random() < 0.05:
                dict_data[k] = None
                null_counts[k] += 1  # Увеличиваем счетчик для этого столбца


        # Создаем кортеж данных и добавляем его в итоговый список
        row_data = (i + 1, dict_data['name'], dict_data['email'], dict_data['city'],\
                    dict_data['age'], dict_data['salary'], dict_data['registration_date'])

        data.append(row_data)

    return data



In [21]:
if __name__ == '__main__':
# вариант ввода числа с помощью input
#    num = int(input('Введите число генерируемых данных (количество строк): '))
#    data = generate_data(num)

# вариант с использованием библиотеки argparse, через аргумент
    # Чтение аргументов командной строки
    parser = argparse.ArgumentParser(description="Генерация синтетических данных.")
    parser.add_argument('-r', '--rows', help='Число генерируемых данных (количество строк)',
                        nargs='?', type=int, default=1000)
    # Разбор аргументов, игнорируя незнакомые
    args, unknown = parser.parse_known_args()
    # Получаем значение из аргумента rows (если он был передан), либо используем дефолтное значение
    num = args.rows
    data = generate_data(num)

In [22]:
    spark = SparkSession.builder \
        .appName("generator_csv") \
        .config("spark.master", "local[*]") \
        .getOrCreate()

In [23]:
    schema = StructType([
        StructField('id', IntegerType(), False),
        StructField('name', StringType()),
        StructField('email', StringType()),
        StructField('city', StringType()),
        StructField('age', IntegerType()),
        StructField('salary', IntegerType()),
        StructField('registration_date', DateType())
    ])

    df = spark.createDataFrame(data=data, schema=schema)
    count_data = df.count()

In [24]:
    df.printSchema()

root
 |-- id: integer (nullable = false)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- city: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- registration_date: date (nullable = true)



In [32]:
""" #Проверка нулевых данных
# Data Frame с NULL значениями
df_empty_values = df.filter(df['name'].isNull() | df['email'].isNull() \
                            | df['city'].isNull() | df['age'].isNull()\
                            | df['salary'].isNull() | df['registration_date'].isNull())

# количество NULL значений по столбцам
cnt_null_name = df_empty_values.filter(col('name').isNull()).count()
cnt_null_email = df_empty_values.filter(col('email').isNull()).count()
cnt_null_city = df_empty_values.filter(col('city').isNull()).count()
cnt_null_age = df_empty_values.filter(col('age').isNull()).count()
cnt_null_salary = df_empty_values.filter(col('salary').isNull()).count()
cnt_null_reg_date = df_empty_values.filter(col('registration_date').isNull()).count()

# Функция для проверки условия на превышение NULL значений по столбцам
def check_null_percentage(column_name, cnt_null, count_data):
    null_percentage = (cnt_null / count_data) * 100
    if null_percentage > 5 and null_percentage < 6:
        print(f'количество NULL в столбце "{column_name}" незначительно превышает 5% и равна {null_percentage:.2f}%')
    elif null_percentage > 6:
        print(f'количество NULL в столбце "{column_name}" ЗНАЧИТЕЛЬНО превышает 5% и равна {null_percentage:.2f}%')
    else:
        print(f'количество NULL в столбце "{column_name}" не превышает 5% и равна {null_percentage:.2f}%')

# Проверка для каждого столбца
check_null_percentage('name', cnt_null_name, count_data)
check_null_percentage('email', cnt_null_email, count_data)
check_null_percentage('city', cnt_null_city, count_data)
check_null_percentage('age', cnt_null_age, count_data)
check_null_percentage('salary', cnt_null_salary, count_data)
check_null_percentage('registration_date', cnt_null_reg_date, count_data)
"""

количество NULL в столбце "name" не превышает 5% и равна 4.70%
количество NULL в столбце "email" не превышает 5% и равна 4.70%
количество NULL в столбце "city" не превышает 5% и равна 4.00%
количество NULL в столбце "age" незначительно превышает 5% и равна 5.10%
количество NULL в столбце "salary" не превышает 5% и равна 4.50%
количество NULL в столбце "registration_date" незначительно превышает 5% и равна 5.10%


In [33]:
output_path = '/content/sample_data'
df.coalesce(1).write.csv(output_path, header=True, mode="overwrite")
filename = rename_csv(output_path)
print(f'Было сгенерировано {count_data} строк данных, \
которые были записаны в файл \'{filename}\'')

Было сгенерировано 1000 строк данных, которые были записаны в файл '/content/sample_data/2025-02-20-dev.csv'


In [34]:
    spark.stop()

In [ ]:
# ВЕСЬ КОД НИЖЕ

In [35]:
import random
import datetime
import os
import argparse
from transliterate import translit
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql.functions import col, when, count

# Список из 25 имен
prepared_list_names = [
    "Алексей", "Ольга", "Дмитрий", "Мария", "Иван",
    "Анна", "Сергей", "Елена", "Андрей", "Наталья",
    "Владимир", "Татьяна", "Михаил", "Светлана", "Кирилл",
    "Юлия", "Александр", "Екатерина", "Николай", "Ирина",
    "Василий", "Евгений", "Людмила", "Павел", "Роман"
]

# Список из 40 городов
prepared_list_cities = [
    "Москва", "Санкт-Петербург", "Новосибирск", "Екатеринбург", "Казань",
    "Нижний Новгород", "Челябинск", "Самара", "Омск", "Ростов-на-Дону",
    "Уфа", "Красноярск", "Воронеж", "Пермь", "Волгоград",
    "Краснодар", "Тольятти", "Ижевск", "Ульяновск", "Барнаул",
    "Тюмень", "Иркутск", "Саратов", "Хабаровск", "Ярославль",
    "Владивосток", "Махачкала", "Томск", "Оренбург", "Кемерово",
    "Новокузнецк", "Рязань", "Астрахань", "Пенза", "Липецк",
    "Тула", "Киров", "Брянск", "Чебоксары", "Калининград"
]

current_date = datetime.date.today()


# ф-ция генерации даты рождения
def generate_birth_date(age: int) -> datetime.date:
    # Расчет даты рождения для случайного возраста
    random_date_of_birth = current_date - datetime.timedelta(days=age * 365)

    return random_date_of_birth

# ф-ция генерации даты регистрации.
# Ограничения:
# регистрация возможна после достижения человеком пяти лет и до текущей даты.
def generate_registration_date(birth_date: datetime.date) -> datetime.date:
    # toordinal() переводит дату в дни, а fromordinal() обратно в дату
    start_date = birth_date.toordinal() + 5*365
    end_date = current_date.toordinal()
    random_registr_date = datetime.date \
        .fromordinal(random.randint(start_date, end_date))

    return random_registr_date

# ф-ции выбора случайного имени и города из отфильтрованного списка
def random_name() -> str:
  return random.choice([name for name in prepared_list_names if len(name) >= 5])

def random_city() -> str:
  return random.choice([city for city in prepared_list_cities if len(city) >= 7])


# ф-ция удаляет все файлы, кроме .csv и переименовывает его в формат "текущий год-месяц-день-dev.csv"
def rename_csv(output_path: str) -> str:
    files_list = os.listdir(output_path)
    new_filename = output_path + f'/{current_date}-dev.csv'

    for file in files_list:
        if file.startswith('part-') and file.endswith('.csv'):
            old_name = output_path + '/' + file
            os.rename(old_name, new_filename)
        else:
            os.remove(os.path.join(output_path, file))

    return new_filename


# ф-ция генерации данных
def generate_data(num: int) -> list[tuple]:
    data = []

    # Рассчитываем количество NULL значений (не более 5% в каждом столбце)
    number_null_in_table = int(num * 0.05)

    # Для каждого столбца создаем свой счетчик
    null_counts = {
        'name': 0,
        'email': 0,
        'city': 0,
        'age': 0,
        'salary': 0,
        'registration_date': 0
    }

    # Генерация данных
    for i in range(0, num):
        name = random_name()
        city = random_city()
        age = random.randint(18, 95)  # Генерация случайного возраста от 18 до 95 лет
        birth_date = generate_birth_date(age)
        registration_date = generate_registration_date(birth_date)
        # во время преобразования символов из ru в en translit заменяет "ь" на "'' (Ольга -> Ol'ga)
        # с помощью .replace("'", "") убираем "'" (Ольга -> Olga)
        email = translit(name.lower(), 'ru', reversed=True).replace("'", "") \
                + str(birth_date.year) + '_' + str(birth_date.day) + '@' \
                + random.choice(['ru', 'com'])
        salary = random.choice(range(100, 500)) * 10 ** 3

        # промежуточная запись в словарь
        dict_data = {
            'name': name,
            'city': city,
            'age': age,
            'registration_date': registration_date,
            'email': email,
            'salary': salary
        }

        # Проверяем для каждого ключа (столбца) условие для вставки NULL значений
        for k, v in dict_data.items():
            # 1. счетчик для столбца <= допустимому кол-ву NULL значений по столбцу
            # 2. вероятность вставки NULL (если random.random() возвращает число меньше 0.05)
            if null_counts[k] <= number_null_in_table and random.random() < 0.05:
                dict_data[k] = None
                null_counts[k] += 1  # Увеличиваем счетчик для этого столбца

        # Создаем кортеж данных и добавляем его в итоговый список
        row_data = (i + 1, dict_data['name'], dict_data['email'], dict_data['city'], \
                    dict_data['age'], dict_data['salary'], dict_data['registration_date'])

        data.append(row_data)

    return data

if __name__ == '__main__':
# вариант ввода числа с помощью input
#    num = int(input('Введите число генерируемых данных (количество строк): '))
#    data = generate_data(num)

# вариант с использованием библиотеки argparse, через аргумент
    # Чтение аргументов командной строки
    parser = argparse.ArgumentParser(description="Генерация синтетических данных.")
    parser.add_argument('-r', '--rows', help='Число генерируемых данных (количество строк)',
                        nargs='?', type=int, default=1000)
    # Разбор аргументов, игнорируя незнакомые
    args, unknown = parser.parse_known_args()
    # Получаем значение из аргумента rows (если он был передан), либо используем дефолтное значение
    num = args.rows
    data = generate_data(num)

    spark = SparkSession.builder \
        .appName("generator_csv") \
        .config("spark.master", "local[*]") \
        .getOrCreate()

    schema = StructType([
        StructField('id', IntegerType(), False),
        StructField('name', StringType()),
        StructField('email', StringType()),
        StructField('city', StringType()),
        StructField('age', IntegerType()),
        StructField('salary', IntegerType()),
        StructField('registration_date', DateType())
    ])

    df = spark.createDataFrame(data=data, schema=schema)
    count_data = df.count()


    """#Проверка нулевых данных
    # Data Frame с NULL значениями
    df_empty_values = df.filter(df['name'].isNull() | df['email'].isNull() \
                                | df['city'].isNull() | df['age'].isNull()\
                                | df['salary'].isNull() | df['registration_date'].isNull())

    # количество NULL значений по столбцам
    cnt_null_name = df_empty_values.filter(col('name').isNull()).count()
    cnt_null_email = df_empty_values.filter(col('email').isNull()).count()
    cnt_null_city = df_empty_values.filter(col('city').isNull()).count()
    cnt_null_age = df_empty_values.filter(col('age').isNull()).count()
    cnt_null_salary = df_empty_values.filter(col('salary').isNull()).count()
    cnt_null_reg_date = df_empty_values.filter(col('registration_date').isNull()).count()

    # Функция для проверки условия на превышение NULL значений по столбцам
    def check_null_percentage(column_name, cnt_null, count_data):
        null_percentage = (cnt_null / count_data) * 100
        if null_percentage > 5 and null_percentage < 6:
            print(f'количество NULL в столбце "{column_name}" незначительно превышает 5% и равна {null_percentage:.2f}%')
        elif null_percentage > 6:
            print(f'количество NULL в столбце "{column_name}" ЗНАЧИТЕЛЬНО превышает 5% и равна {null_percentage:.2f}%')
        else:
            print(f'количество NULL в столбце "{column_name}" не превышает 5% и равна {null_percentage:.2f}%')

    # Проверка для каждого столбца
    check_null_percentage('name', cnt_null_name, count_data)
    check_null_percentage('email', cnt_null_email, count_data)
    check_null_percentage('city', cnt_null_city, count_data)
    check_null_percentage('age', cnt_null_age, count_data)
    check_null_percentage('salary', cnt_null_salary, count_data)
    check_null_percentage('registration_date', cnt_null_reg_date, count_data)"""


    output_path = '/content/sample_data'
    df.coalesce(1).write.csv(output_path, header=True, mode="overwrite")
    filename = rename_csv(output_path)
    print(f'Было сгенерировано {count_data} строк данных, \
которые были записаны в файл \'{filename}\'')

    spark.stop()

Было сгенерировано 1000 строк данных, которые были записаны в файл '/content/sample_data/2025-02-20-dev.csv'
